<a href="https://colab.research.google.com/github/Bryan-Az/TransformerLLM-Finetuning/blob/main/NanoGPT_TransformerLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A 'Nano' Generative Pre-trained Transformer LLM
This is an experiment where I create and train a mini GPT model from scratch. It will be using a ebook of my choice as its' training data.

## Imports and Installs

In [6]:
import gdown

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Extracting, Transforming, and Loading the Ebook Data
The book I chose is "Voices from within the Veil" by W.E.B DuBois. This book was taken from the Project Gutenberg website and is available to use freely. I uploaded it to Google Drive for my own use as to not scrape project gutenberg's website.

In [9]:
def download_file_from_google_drive(glink):
    id = glink.split('/')[-2]
    filepath = None
    try:
        filepath = gdown.download(id=id, quiet=False)
    except Exception as e:
        print(f"Error downloading file: {e}")
    ebook = open(filepath, 'r').readlines()
    return ''.join(ebook)

In [11]:
voices_within_veil_glink = 'https://drive.google.com/file/d/1Ah7n0r-bjW8nbTdc82C8_9_4l3c9lOql/view?usp=sharing'
voices_within_veil = download_file_from_google_drive(voices_within_veil_glink)

Downloading...
From: https://drive.google.com/uc?id=1Ah7n0r-bjW8nbTdc82C8_9_4l3c9lOql
To: /content/Voices_from_within_the_Veil_WEBDuBois.txt
100%|██████████| 419k/419k [00:00<00:00, 89.8MB/s]


In [12]:
print(voices_within_veil[50:305])

from Within the Veil
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project 


In [15]:
chars = sorted(list(set(voices_within_veil))) #chars and vocab size are parameters used within the model and tokenizer
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzàäçéëñô—‘’“”•™﻿
98


In [16]:
# creating a tokenizer using a simple encoding schema
class Tokenizer:
    def __init__(self, chars):
        self.chars = chars
        self.char_to_token = {ch: i for i, ch in enumerate(chars)}
        self.token_to_char = {i: ch for i, ch in enumerate(chars)}

    def encode(self, input_text):
        """Encodes a string into a list of integers.

        Args:
            input_text: The string to encode.

        Returns:
            A list of integers representing the encoded string.
        """
        return [self.char_to_token[c] for c in input_text]

    def decode(self, tokens):
        """Decodes a list of integers into a string.

        Args:
            tokens: The list of integers to decode.

        Returns:
            The decoded string.
        """
        return ''.join([self.token_to_char[i] for i in tokens])

In [17]:
tokenizer = Tokenizer(chars)

In [18]:
# encoding the data
encoded_voices_within_veil = tokenizer.encode(voices_within_veil)

In [22]:
# converting the encoded text into a tensor
encoded_voices_within_veil = torch.tensor(encoded_voices_within_veil)

In [23]:
# saving the ebook into train and test splits
percent_train = 0.9
train_data = encoded_voices_within_veil[:int(percent_train*len(encoded_voices_within_veil))]
test_data = encoded_voices_within_veil[int(percent_train*len(encoded_voices_within_veil)):]

In [24]:
f"Size of train, test splits: {len(train_data), len(test_data)}"

'Size of train, test splits: (369613, 41069)'

In [25]:
# sent the train and test data to gpu device
train_data = train_data.to(device)
test_data = test_data.to(device)

In [26]:
# function to batch the data so that the first N words are 'context' and the final N+1 word is the 'target'.
batch_size = 4 # independent sequences processed in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch; inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size - 3): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [61,  1, 64, 57, 78, 61,  1, 76],
        [82, 57, 76, 65, 71, 70,  1, 71],
        [76, 61,  0,  3, 72, 74, 61, 75]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 1, 64, 57, 78, 61,  1, 76, 71],
        [57, 76, 65, 71, 70,  1, 71, 62],
        [61,  0,  3, 72, 74, 61, 75, 76]], device='cuda:0')
----
when input is [1] the target: 1
when input is [1, 1] the target: 1
when input is [1, 1, 1] the target: 1
when input is [1, 1, 1, 1] the target: 1
when input is [1, 1, 1, 1, 1] the target: 1
when input is [1, 1, 1, 1, 1, 1] the target: 1
when input is [1, 1, 1, 1, 1, 1, 1] the target: 1
when input is [1, 1, 1, 1, 1, 1, 1, 1] the target: 1


## Initializing the Mini Transformer Model

In [27]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [28]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [29]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [30]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [31]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [32]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        idx = torch.clamp(idx, 0, self.token_embedding_table.num_embeddings - 1)
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # Ensure targets are within the valid range for logits
            targets = torch.clamp(targets, 0, C - 1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

## Training the Mini Transformer LLM with Self-Attention for 10,000 Steps

In [33]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [34]:
nanoGPT_model = BigramLanguageModel()
nanoGPT_model = nanoGPT_model.to(device)

In [35]:
# this is the number of parameters in the model per million
print(sum(p.numel() for p in nanoGPT_model.parameters())/1e6, 'M parameters')

# using the pyTorch AdamW optimizer
optimizer = torch.optim.AdamW(nanoGPT_model.parameters(), lr=learning_rate)
eval_interval = 1000

for iter in range(max_iters):

    # evaluate the model and print the loss every 1000 steps
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(nanoGPT_model)
        if iter % eval_interval == 0:
          print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = nanoGPT_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.213986 M parameters
step 0: train loss 4.7520, val loss 4.7472
step 1000: train loss 2.0895, val loss 2.3192
step 2000: train loss 1.8783, val loss 2.1746
step 3000: train loss 1.7980, val loss 2.0762
step 4000: train loss 1.7139, val loss 2.0313
step 5000: train loss 1.6635, val loss 1.9782
step 6000: train loss 1.6308, val loss 1.9866
step 7000: train loss 1.6092, val loss 1.9630
step 8000: train loss 1.5946, val loss 1.9439
step 9000: train loss 1.5648, val loss 1.9124


## Generating Text Completion from the Nano-GPT Model

In [45]:
input = 'Voices from within the'
expected_output = 'Voices from within the Veil'

In [46]:
context = torch.tensor(tokenizer.encode(input), dtype=torch.long, device=device).unsqueeze(0)

In [47]:
output = tokenizer.decode(nanoGPT_model.generate(context, max_new_tokens=20)[0].tolist())

In [48]:
print(f"Input: {input}")
print(f"Expected Output: {expected_output}")
print(f"Model Output: {output}")

Input: Voices from within the
Expected Output: Voices from within the Veil
Model Output: Voices from within the givered ramily line
